# Bibliotecas

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text

# Conexão com MySQL

Informações de conexão do banco de dados

In [ ]:
user = 'root'
password = 'mysqlpwd'
host = 'localhost'
database = 'desafioModulo02'

string_conexao = f'mysql://{user}:{password}@{host}/{database}'
string_conexao

Cria o objeto de conexão

In [ ]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()

# Início da lógica do programa
Faz a leitura do dataset de jogadores

In [ ]:
df = pd.read_csv('data\\dados_jogadores.csv', sep=';', encoding='latin-1')

In [ ]:
# Verifica se existem colunas vazias
df.info()

In [ ]:
# Verifica quais são as colunas que possuem valores vazios e respectivas quantidades
df.isna().sum()

De acordo com as informações acima, existem 3 colunas com informações vazias:<br>
<b>num_vitorias:</b> 8 ocorrências<br>
<b>num_derrotas:</b> 5 ocorrências<br>
<b>total_partidas:</b> 8 ocorrências<br>

Qual foi a lógica para consertar essas informações vazias:<br>
<b>num_vitorias:</b> Pegar o total de partidas e subtrair o número de derrotas;<br>
<b>num_derrotas:</b> Pegar o total de partidas e subtrair o número de vitórias;<br>
<b>total_partidas:</b> Somar a quantidade de vitórias com a quantidade de derrotas;

In [ ]:
# Ajustando o dataset com os campos vazios
df.num_vitorias.fillna(df['total_partidas'] - df['num_derrotas'], inplace=True)
df.num_derrotas.fillna(df['total_partidas'] - df['num_vitorias'], inplace=True)
df.total_partidas.fillna(df['num_vitorias'] + df['num_derrotas'], inplace=True)


In [ ]:
# Verifica novamente se existem informações vazias (todas as colunas devem indicar 0 (zer0))
df.isna().sum()

Teste para o tratamento de data

In [ ]:
data = '03/09/1990'
data_tratada = str(data).split('/')[2] + '/' + str(data).split('/')[1] + '/' + str(data).split('/')[0]
data_tratada

Aplica o tratamento de data para todo o dataframe criando uma nova coluna chamada 'data_nascimento_tratada'

In [ ]:
df['data_nascimento_tratada'] = df['data_nascimento'].str.split('/').str[2] + '/' + df['data_nascimento'].str.split('/').str[1] + '/' + df['data_nascimento'].str.split('/').str[0]

In [ ]:
# Checagem numa amostra para verificar se a data tratada está correta
df[:10]

Lista de países

In [ ]:
lst_pais = list(df.pais.unique())
lst_pais

Lista de jogos

In [ ]:
lst_jogo = list(df.jogo.unique())
lst_jogo

Insere os países no banco de dados

In [ ]:
for pais in lst_pais:
    try:
        query = f"insert into pais (nome_pais) values ('{pais}')"
        conn.execute(text(query))
        print(f"Registro inserido com sucesso: {pais}.")
        
    except Exception as e:
        print(f'Não foi possível inserir o registro {pais}. O erro encontrado foi: {e}.')

Insere os jogos no banco de dados

In [ ]:
for jogo in lst_jogo:
    try:
        query = f"insert into jogo (nome_jogo) values ('{jogo}')"
        conn.execute(text(query))
        print(f"Registro inserido com sucesso: {jogo}.")
        
    except Exception as e:
        print(f'Não foi possível inserir o registro {jogo}. O erro encontrado foi: {e}.')

Insere o dataframe numa tabela stage para tratar dentro do banco de dados

In [ ]:
df.to_sql('stg_jogador', con=conn, schema='desafioModulo02', if_exists='replace', index=False)

In [ ]:
# Confirma todas as transações realizadas no banco de dados
conn.commit()